### Bloco 1: Importação de bibliotecas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time

print("Bibliotecas importadas com sucesso!")

### Bloco 2: Inicialização do WebDriver

In [ ]:
options = Options()
options.add_argument("--start-maximized")  # Abre o navegador maximizado
service = Service(ChromeDriverManager().install())  # Instala o driver automaticamente
driver = webdriver.Chrome(service=service, options=options)
print("WebDriver inicializado com sucesso!")

### Bloco 3: Função de login

In [ ]:
def realizar_login(driver, email, password):
    """
    Realiza o login no Hostinger Mail.
    """
    try:
        print("Abrindo a página de login...")
        driver.get("https://mail.hostinger.com")

        email_field = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.NAME, "_user"))
        )
        email_field.send_keys(email)

        password_field = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.NAME, "_pass"))
        )
        password_field.send_keys(password)

        login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
        )
        login_button.click()

        WebDriverWait(driver, 20).until(
            EC.url_changes("https://mail.hostinger.com")
        )
        print("Login realizado com sucesso!")
    except Exception as e:
        print(f"Erro durante o login: {e}")

realizar_login(driver, "xxx", "xxx")

### Bloco 4: Função para clicar na pasta

In [ ]:
def clicar_na_pasta(driver, nome_pasta):
    """
    Clica na pasta especificada pelo título ou texto visível.
    """
    try:
        print(f"Procurando pela pasta com o título: {nome_pasta} ...")

        # XPath atualizado para localizar pelo título ou texto visível
        xpath = f"//a[@title='{nome_pasta}' or text()='{nome_pasta}']"

        # Aguarda o carregamento da página e a presença do elemento
        pasta_elemento = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )

        # Aguarda até que o elemento seja clicável
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))

        # Role até o elemento e clique
        driver.execute_script("arguments[0].scrollIntoView(true);", pasta_elemento)
        driver.execute_script("arguments[0].click();", pasta_elemento)

        # Aguarda o carregamento da nova página/pasta
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//tr[contains(@class,'message')]"))
        )

        print(f"Pasta com o título '{nome_pasta}' acessada com sucesso.")
    except Exception as e:
        print(f"Erro ao acessar a pasta com o título '{nome_pasta}': {e}")

clicar_na_pasta(driver, "xxx")

### Bloco 5: Processar emails do mês atual com anexos

In [ ]:
def processar_e_baixar_emails_mes_corrente(driver):
    """
    Processa os emails da pasta aberta, verifica se estão no mês atual, abre os emails e baixa os anexos.
    """
    try:
        print("Procurando emails do mês corrente com anexos...")

        # Obtém a data atual
        hoje = datetime.datetime.now()
        inicio_mes = hoje.replace(day=1)

        # Localiza todos os emails na lista
        emails = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//tr[contains(@class,'message')]"))
        )
        print(f"E-mails encontrados: {len(emails)}")

        for index, email in enumerate(emails):
            try:
                print(f"Processando email {index + 1}/{len(emails)}...")

                # Verifica se o email contém anexo
                tem_anexo = email.find_elements(By.XPATH, ".//span[@class='attachment' and @title='Com anexo']")
                if not tem_anexo:
                    print(f"Email {index + 1} não contém anexos. Pulando...")
                    continue

                # Verifica se a data do email é do mês atual
                try:
                    data_element = email.find_element(By.XPATH, ".//span[contains(@class,'date')]")
                    data_texto = data_element.text.strip()
                    print(f"Data do email (texto bruto): {data_texto}")
                except Exception as e:
                    print(f"Erro ao localizar a data no email {index + 1}: {e}")
                    continue

                data_email = interpretar_data_email(data_texto)
                print(f"Data convertida: {data_email}, Início do mês: {inicio_mes}, Hoje: {hoje}")

                if data_email is None or data_email < inicio_mes or data_email > hoje:
                    print(f"Email {index + 1} fora do intervalo do mês corrente. Pulando...")
                    continue

                print(f"Email válido encontrado: Data: {data_email}, contém anexo.")

                # Clica no email
                print(f"Tentando clicar no email {index + 1}...")
                driver.execute_script("arguments[0].scrollIntoView(true);", email)
                email.click()

                # Aguarda a lista de anexos no email
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//ul[@id='attachment-list']"))
                )
                print(f"Email {index + 1} aberto com sucesso.")

                # Localiza os anexos e clica para baixar
                anexos = driver.find_elements(By.XPATH, "//ul[@id='attachment-list']//a[contains(@class, 'filename')]")
                if not anexos:
                    print(f"Não foram encontrados anexos no email {index + 1}.")
                    driver.back()
                    continue

                for anexo in anexos:
                    print(f"Baixando anexo: {anexo.text}")
                    anexo.click()
                    time.sleep(3)  # Aguarda o download

                # Volta para a lista de emails
                driver.back()
                print(f"Retornando à lista de emails após processar o email {index + 1}...")
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//tr[contains(@class,'message')]"))
                )

            except Exception as e:
                print(f"Erro ao processar email {index + 1}: {e}")
    except Exception as e:
        print(f"Erro ao verificar emails: {e}")

### Execução do processamento
processar_e_baixar_emails_mes_corrente(driver)
